In [2]:
import pandas as pd

In [3]:
df=pd.read_csv('Datathon_TrainData_org.csv')

In [4]:
df

,Country,Date,Speaker,Headline,Text_of_Speech,Designation,Running President/PM,Speech Link,Emotion,Context
0,Russia,24-01-2000,Vladimir Putin,Speech at a conference of the heads of republi...,"Esteemed Mr Lebedev,\n\nLadies and Gentlemen:\...",President,Vladimir Putin,http://en.kremlin.ru/events/president/transcri...,UPSET,OTHERS
1,USA,09-09-2021 00:00,Joe Biden,Remarks by President Biden on Fighting the COV...,"THE PRESIDENT: Good evening, my fellow Americ...",President,Joe Biden,https://www.whitehouse.gov/briefing-room/speec...,OPTIMISM,DEVELOPMENT
2,Russia,24-12-2004,Vladimir Putin,Excerpts from the Opening Speech at a Session ...,"Good afternoon, dear colleagues,\n\nOur meetin...",President,Vladimir Putin,http://en.kremlin.ru/events/president/transcri...,NEUTRAL,DEVELOPMENT
3,USA,14-08-2014,Barack Obama,\nStatement by the President,"Good afternoon, everybody. This sound system i...",President,Barack Obama,https://obamawhitehouse.archives.gov/the-press...,NEUTRAL,INTERNATIONAL AFFAIRS
4,USA,26-03-2021 00:00,Joe Biden,Statement by President Biden on the Attack on ...,More Americans voted in the 2020 elections tha...,President,Joe Biden,https://www.whitehouse.gov/briefing-room/speec...,UPSET,OTHERS
...,...,...,...,...,...,...,...,...,...,...
1612,United Kingdom,24-Jul-19,Boris Johnson,Boris Johnson's first speech as Prime Minister...,Good afternoon I have just been to see Her Maj...,Prime Minister,Boris Johnson,https://www.gov.uk/government/speeches/boris-j...,OPTIMISM,DEVELOPMENT
1613,USA,01-02-2021 00:00,Joe Biden,Statement by President Joe Biden On Black Hist...,"This February, during Black History Month, I c...",President,Joe Biden,https://www.whitehouse.gov/briefing-room/state...,OPTIMISM,OTHERS
1614,USA,30-09-2015,Barack Obama,\nRemarks by the President Meeting with State ...,Thank you so much. Everybody have a seat. We...,President,Barack Obama,https://obamawhitehouse.archives.gov/the-press...,NEUTRAL,NATIONALISM
1615,China,23-06-2021 00:00,H.E. Wang Yi,Let Us Strengthen Confidence and Solidarity an...,"Colleagues,\nFriends,\n\nIn 2013, President Xi...",State Councilor and Foreign Minister of the Pe...,Xi Jinping,https://www.fmprc.gov.cn/mfa_eng/wjdt_665385/z...,JOY,DEVELOPMENT


In [5]:
import pandas as pd
import numpy as np
import re
from datetime import datetime
from sklearn.preprocessing import LabelEncoder
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')


[nltk_data] Downloading package punkt to C:\Users\devayani
[nltk_data]     ponduru\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [6]:
df.dropna(inplace=True)

In [7]:
df['Date'] = pd.to_datetime(df['Date'])

In [8]:
df['Headline'] = df['Headline'].str.lower()

In [9]:
df['Headline'] = df['Headline'].apply(nltk.word_tokenize)

In [10]:
df['Headline'] = df['Headline'].apply(lambda tokens: [re.sub(r'[^a-zA-Z0-9]', '', token) for token in tokens])

In [11]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
if 'Headline' in df.columns:
    stop_words = set(stopwords.words('english'))
    def preprocess_text(text):
        if isinstance(text, str):
            tokens = nltk.word_tokenize(text)
            tokens = [token for token in tokens if token not in stop_words]
            return ' '.join(tokens)
        else:
            return text
    df['Headline'] = df['Headline'].apply(preprocess_text)

[nltk_data] Downloading package stopwords to C:\Users\devayani
[nltk_data]     ponduru\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [12]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
nltk.download('stopwords')
if 'Headline' in df.columns:
    stop_words = set(stopwords.words('english'))
    stemmer = PorterStemmer()
    def preprocess_text(text):
        if isinstance(text, str):
            tokens = nltk.word_tokenize(text)
            tokens = [token for token in tokens if token not in stop_words]
            stemmed_tokens = [stemmer.stem(token) for token in tokens]
            return ' '.join(stemmed_tokens)
        else:
            return text
    df['Headline'] = df['Headline'].apply(preprocess_text)

[nltk_data] Downloading package stopwords to C:\Users\devayani
[nltk_data]     ponduru\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [13]:
df['Headline'] = df['Headline'].apply(lambda tokens: [token if not token.isdigit() else 'NUM' for token in tokens])

In [14]:
df['Headline'] = df['Headline'].apply(lambda tokens: ' '.join(tokens))

In [15]:
df.to_csv('preprocessed_datathon_traindata.csv', index=False)

In [16]:
le = LabelEncoder()
df['Emotion'] = le.fit_transform(df['Emotion'])

In [25]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
df = pd.read_csv('preprocessed_datathon_traindata.csv')
X = df['Text_of_Speech']
y = df['Emotion']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)
tfidf_vectorizer = TfidfVectorizer(max_features=1000)
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)
model = MultinomialNB()
model.fit(X_train_tfidf, y_train)
y_pred = model.predict(X_test_tfidf)
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
classification_rep = classification_report(y_test, y_pred)
print(f"Accuracy: {accuracy}")
print("Confusion Matrix:")
print(conf_matrix)
print("Classification Report:")
print(classification_rep)

Accuracy: 0.37267080745341613
Confusion Matrix:
[[21 10  5  0]
 [13 26 10  0]
 [13 23 12  0]
 [10 13  4  1]]
Classification Report:
              precision    recall  f1-score   support

         JOY       0.37      0.58      0.45        36
     NEUTRAL       0.36      0.53      0.43        49
    OPTIMISM       0.39      0.25      0.30        48
       UPSET       1.00      0.04      0.07        28

    accuracy                           0.37       161
   macro avg       0.53      0.35      0.31       161
weighted avg       0.48      0.37      0.33       161



In [26]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Load the data
df = pd.read_csv('preprocessed_datathon_traindata.csv')

# Split the data into train and test sets
X = df['Text_of_Speech']
y = df['Emotion']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

# Use a more advanced vectorizer
tfidf_vectorizer = TfidfVectorizer(max_features=10000, ngram_range=(1, 2))
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

# Train a Multinomial Naive Bayes model
model = MultinomialNB(alpha=0.1)  # Adjust alpha based on hyperparameter tuning
model.fit(X_train_tfidf, y_train)

# Make predictions
y_pred = model.predict(X_test_tfidf)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
classification_rep = classification_report(y_test, y_pred)

print(f"Accuracy: {accuracy}")
print("Confusion Matrix:")
print(conf_matrix)
print("Classification Report:")
print(classification_rep)


Accuracy: 0.391304347826087
Confusion Matrix:
[[20 10  5  1]
 [11 22 12  4]
 [11 22 14  1]
 [ 9  9  3  7]]
Classification Report:
              precision    recall  f1-score   support

         JOY       0.39      0.56      0.46        36
     NEUTRAL       0.35      0.45      0.39        49
    OPTIMISM       0.41      0.29      0.34        48
       UPSET       0.54      0.25      0.34        28

    accuracy                           0.39       161
   macro avg       0.42      0.39      0.38       161
weighted avg       0.41      0.39      0.38       161



In [19]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import GridSearchCV

# Load the data
df = pd.read_csv('preprocessed_datathon_traindata.csv')

# Split the data into train and test sets
X = df['Text_of_Speech']
y = df['Emotion']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Use a more advanced vectorizer
tfidf_vectorizer = TfidfVectorizer(max_features=10000, ngram_range=(1, 2))
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

# Perform hyperparameter tuning using GridSearchCV
param_grid = {'alpha': [0.01, 0.1, 1.0, 10.0]}  # Adjust the alpha values as needed
grid_search = GridSearchCV(MultinomialNB(), param_grid, cv=5)
grid_search.fit(X_train_tfidf, y_train)
best_alpha = grid_search.best_params_['alpha']

# Train a Multinomial Naive Bayes model with the best alpha
model = MultinomialNB(alpha=best_alpha)
model.fit(X_train_tfidf, y_train)

# Make predictions
y_pred = model.predict(X_test_tfidf)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
classification_rep = classification_report(y_test, y_pred)

print(f"Best Alpha: {best_alpha}")
print(f"Accuracy: {accuracy}")
print("Confusion Matrix:")
print(conf_matrix)
print("Classification Report:")
print(classification_rep)


Best Alpha: 0.1
Accuracy: 0.46105919003115264
Confusion Matrix:
[[46 20 10  2]
 [18 52 21  4]
 [19 38 34  1]
 [14 19  7 16]]
Classification Report:
              precision    recall  f1-score   support

         JOY       0.47      0.59      0.53        78
     NEUTRAL       0.40      0.55      0.46        95
    OPTIMISM       0.47      0.37      0.41        92
       UPSET       0.70      0.29      0.41        56

    accuracy                           0.46       321
   macro avg       0.51      0.45      0.45       321
weighted avg       0.49      0.46      0.45       321



In [20]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import GridSearchCV

# Load the data
df = pd.read_csv('preprocessed_datathon_traindata.csv')

# Split the data into train and test sets
X = df['Text_of_Speech']
y = df['Emotion']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Use a more advanced vectorizer
tfidf_vectorizer = TfidfVectorizer(max_features=10000, ngram_range=(1, 2))
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

# Hyperparameter tuning using GridSearchCV for Random Forest
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

grid_search = GridSearchCV(RandomForestClassifier(random_state=42), param_grid, cv=5)
grid_search.fit(X_train_tfidf, y_train)
best_rf_model = grid_search.best_estimator_

# Make predictions
y_pred = best_rf_model.predict(X_test_tfidf)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
classification_rep = classification_report(y_test, y_pred)

print(f"Best Random Forest Model: {best_rf_model}")
print(f"Accuracy: {accuracy}")
print("Confusion Matrix:")
print(conf_matrix)
print("Classification Report:")
print(classification_rep)


Best Random Forest Model: RandomForestClassifier(max_depth=20, min_samples_split=10, n_estimators=300,
                       random_state=42)
Accuracy: 0.4735202492211838
Confusion Matrix:
[[50 15 13  0]
 [19 53 23  0]
 [13 33 46  0]
 [18 24 11  3]]
Classification Report:
              precision    recall  f1-score   support

         JOY       0.50      0.64      0.56        78
     NEUTRAL       0.42      0.56      0.48        95
    OPTIMISM       0.49      0.50      0.50        92
       UPSET       1.00      0.05      0.10        56

    accuracy                           0.47       321
   macro avg       0.60      0.44      0.41       321
weighted avg       0.56      0.47      0.44       321



In [22]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.feature_selection import SelectKBest, chi2

8

# Split the data into train and test sets
X = df['Text_of_Speech']
y = df['Emotion']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Use a different vectorizer or change vectorization parameters
tfidf_vectorizer = TfidfVectorizer(max_features=5000, ngram_range=(1, 2))
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

# Perform feature selection (SelectKBest with chi-squared test)
selector = SelectKBest(chi2, k=2000)  # Adjust the number of features (k) as needed
X_train_tfidf = selector.fit_transform(X_train_tfidf, y_train)
X_test_tfidf = selector.transform(X_test_tfidf)

# Train a Multinomial Naive Bayes model
model = MultinomialNB(alpha=0.1)  # You can adjust alpha if needed
model.fit(X_train_tfidf, y_train)

# Make predictions
y_pred = model.predict(X_test_tfidf)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
classification_rep = classification_report(y_test, y_pred)

print(f"Accuracy: {accuracy}")
print("Confusion Matrix:")
print(conf_matrix)
print("Classification Report:")
print(classification_rep)


Accuracy: 0.42679127725856697
Confusion Matrix:
[[42 21 14  1]
 [21 52 19  3]
 [21 38 31  2]
 [17 18  9 12]]
Classification Report:
              precision    recall  f1-score   support

         JOY       0.42      0.54      0.47        78
     NEUTRAL       0.40      0.55      0.46        95
    OPTIMISM       0.42      0.34      0.38        92
       UPSET       0.67      0.21      0.32        56

    accuracy                           0.43       321
   macro avg       0.48      0.41      0.41       321
weighted avg       0.46      0.43      0.42       321



In [24]:
from sklearn.metrics import confusion_matrix  
cm= confusion_matrix(y_test, y_pred) 
print('confusion matrix:\n',cm)

accurecy=(cm[0][0] + cm[1][1])/(cm[0][0]+ cm[0][1]+cm[1][0]+cm[1][1])
print("Accurecy:\n",accurecy)

confusion matrix:
 [[42 21 14  1]
 [21 52 19  3]
 [21 38 31  2]
 [17 18  9 12]]
Accurecy:
 0.6911764705882353
